In [6]:
# Importowanie wymaganych bibliotek
import cv2  # OpenCV - biblioteka do przetwarzania obrazu
import dlib  # Biblioteka do detekcji twarzy i analizy punktów charakterystycznych
import numpy as np  # Obsługa operacji na macierzach
import imutils  # Pomocnicze operacje na obrazach (np. zmiana rozmiaru)
import tensorflow as tf  # Biblioteka do uczenia maszynowego (nie jest używana w kodzie)
from tensorflow.keras.models import load_model  # Ładowanie modeli sieci neuronowych (ale nie jest używane)
import matplotlib.pyplot as plt  # Biblioteka do wizualizacji
from scipy.spatial import distance as dist  # Obliczanie odległości euklidesowej

# Inicjalizacja detektora twarzy
detector = dlib.get_frontal_face_detector()

# Wczytanie modelu predykcji punktów charakterystycznych twarzy (oczy, usta, nos)
predictor = dlib.shape_predictor("C:\\Users\\Administrator\\projektzmeczenie\\shape_predictor_68_face_landmarks.dat")

# Inicjalizacja kamery (0 oznacza domyślną kamerę)
cap = cv2.VideoCapture(0)

# Funkcja obliczająca Eye Aspect Ratio (EAR) - wskaźnik zamknięcia oczu
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])  # Odległość między górną i dolną powieką (wewnętrzna część oka)
    B = dist.euclidean(eye[2], eye[4])  # Odległość między górną i dolną powieką (zewnętrzna część oka)
    C = dist.euclidean(eye[0], eye[3])  # Odległość między kącikami oka
    return (A + B) / (2.0 * C)  # Wskaźnik EAR - im niższy, tym bardziej zamknięte oczy

# Parametry dla detekcji zmęczenia
EYE_AR_THRESH = 0.25  # Próg zamknięcia oczu - jeśli EAR jest niższe, oczy są zamknięte
EYE_AR_CONSEC_FRAMES = 20  # Liczba klatek, po których uznamy senność kierowcy

# Licznik mrugnięć i flaga ostrzeżenia
blink_counter = 0
drowsy_alert = False  # Flaga wykrycia senności

while True:
    # Pobranie klatki obrazu z kamery
    ret, frame = cap.read()
    if not ret:  # Jeśli nie udało się pobrać klatki, przerwij pętlę
        break

    # Konwersja obrazu na skalę szarości (lepsza detekcja)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Wykrywanie twarzy w obrazie
    faces = detector(gray)

    for face in faces:
        # Pobranie punktów charakterystycznych twarzy (oczy, nos, usta)
        landmarks = predictor(gray, face)

        # Pobranie współrzędnych punktów dla lewego oka (indeksy 36-41)
        left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])

        # Pobranie współrzędnych punktów dla prawego oka (indeksy 42-47)
        right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])

        # Obliczenie wskaźnika EAR dla obu oczu
        left_EAR = eye_aspect_ratio(left_eye)
        right_EAR = eye_aspect_ratio(right_eye)
        EAR = (left_EAR + right_EAR) / 2.0  # Średnia wartość EAR dla obu oczu

        # Sprawdzenie, czy oczy są zamknięte (EAR poniżej progu)
        if EAR < EYE_AR_THRESH:
            blink_counter += 1  # Zwiększenie licznika zamkniętych oczu
        else:
            # Jeśli oczy były zamknięte przez określoną liczbę klatek, uznajemy senność
            if blink_counter >= EYE_AR_CONSEC_FRAMES:
                drowsy_alert = True
            blink_counter = 0  # Zerowanie licznika mrugnięć

    # Wyświetlenie statusu senności na ekranie
    cv2.putText(frame, f"Sleepy: {drowsy_alert}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Wyświetlenie obrazu z kamery
    cv2.imshow("Kamera", frame)

    # Przerwanie działania programu po naciśnięciu 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Zwolnienie kamery i zamknięcie okien
cap.release()
cv2.destroyAllWindows()


In [19]:
!git add .
